<a href="https://colab.research.google.com/github/UiinKim/UiinKim/blob/main/%EB%B2%A1%ED%84%B0%EC%9D%98_%EC%9C%A0%EC%82%AC%EB%8F%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



*   두 벡터 A, B에 대한 코사인 유사도 = dot(A,B)/(norm(A)*norm(B))



In [99]:
docs=['저는 사과 좋아요',
      '저는 바나나 좋아요',
      '저는 바나나 좋아요 저는 바나나 좋아요']

In [100]:
!pip install konlpy

In [101]:
from konlpy.tag import Okt
okt=Okt()

vocab=list(set(word for doc in docs for word in doc.split()))
vocab.sort()
#단어사전 만들기

In [102]:
import pandas as pd
from math import log

n=len(vocab)

def tf(t, d):
  return d.count(t)

def idf(t):
  df=0
  for doc in docs:
    if t in doc:
      df+=1
    #df+= t in doc
  return log(n/(1+df))

def tfidf(t, d):
  return tf(t,d)*idf(t)
#tf, idf, tf-idf 만들기

In [103]:
result=[]
for doc in docs:
  result.append([])
  d=doc
  for word in vocab:
    t=word
    result[-1].append(tf(t, d))
tf_=pd.DataFrame(result, columns=vocab)
tf_
#tf 데이터 프레임 만들기

,바나나,사과,저는,좋아요
0,0,1,1,1
1,1,0,1,1
2,2,0,2,2


In [104]:
result=[]
for doc in docs:
  d=doc
  result.append([])
  for word in vocab:
    t=word
    result[-1].append(tfidf(t, d))
tfidf_=pd.DataFrame(result, columns=vocab)
tfidf_
#tfidf 데이터프레임 만들기

,바나나,사과,저는,좋아요
0,0.000000,0.693147,0.0,0.0
1,0.287682,0.000000,0.0,0.0
2,0.575364,0.000000,0.0,0.0


In [105]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A,B):
  return dot(A,B)/(norm(A)*norm(B))

In [106]:
text=np.array(tf_)

In [107]:
print('문서 1과 문서 2의 유사도 : ', cos_sim(text[0], text[1]))
print('문서 1과 문서 3의 유사도 : ', cos_sim(text[0], text[2]))
print('문서 2과 문서 3의 유사도 : ', cos_sim(text[1], text[2]))
#유클리드 유사도는 문서의 길이에 영향을 받지만 코사인 유사도는 벡터의 방향에 초점을 둔다.

문서 1과 문서 2의 유사도 :  0.6666666666666667
문서 1과 문서 3의 유사도 :  0.6666666666666667
문서 2과 문서 3의 유사도 :  1.0000000000000002


캐글에서 코랩으로 데이터셋 가져오는 과정

In [108]:
!pip install -q kaggle

In [109]:
!pip install kaggle --upgrade

In [110]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle (2).json': b'{"username":"kimuiin","key":"14d4807834488ef54db9dfdbc855736b"}'}

In [111]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [112]:
!ls -1ha kaggle.json

kaggle.json


In [113]:
!kaggle datasets download -d kimuiin/moviesmetadata

moviesmetadata.zip: Skipping, found more recently modified local copy (use --force to force download)


In [114]:
!unzip moviesmetadata.zip

Archive:  moviesmetadata.zip
replace movies_metadata.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [115]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data=pd.read_csv('movies_metadata.csv', low_memory=False)
data.head(2)

#여기서 사용할 내용은 title과 overview(줄거리)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


유사도를 이용한 추천시스템 구현하기

In [116]:
data=data.head(20000)
#overview를 이용해 비슷한 단어들을 가진 줄거리를 추천한다.
#tfidf는 결측치가 있으면 오류가 발생한다.
print('overview 열의 결측값 수 : ', data['overview'].isnull().sum())

overview 열의 결측값 수 :  135


In [117]:
data['overview']=data['overview'].fillna('')
#결측치를 빈 value로 채운다.

In [118]:
tfidf=TfidfVectorizer(stop_words='english')
#영어의 불용어를 제외한다.
tfidf_matrix=tfidf.fit_transform(data['overview'])
#overview의 토큰화 및 벡터화를 시키고 tfidf를 구한다.
print('TF-IDF 행렬의 크기 :', tfidf_matrix.shape)
print(tfidf_matrix)

TF-IDF 행렬의 크기 : (20000, 47487)
  (0, 11645)	0.13942792746208266
  (0, 2811)	0.1524688231743182
  (0, 24325)	0.10272285889754121
  (0, 14393)	0.10323587624822204
  (0, 12920)	0.13359137839416452
  (0, 30780)	0.10362949000778948
  (0, 37778)	0.13329592929044795
  (0, 7974)	0.13030575543855816
  (0, 32424)	0.13359137839416452
  (0, 19047)	0.10485920860731182
  (0, 32291)	0.09637786759647918
  (0, 25229)	0.12052668500637997
  (0, 1254)	0.1489611323980068
  (0, 37166)	0.11699407550916606
  (0, 24759)	0.19553422928903877
  (0, 6290)	0.5036643709879097
  (0, 5775)	0.10961099833635705
  (0, 4736)	0.12600776612424489
  (0, 36185)	0.11514050233158798
  (0, 18727)	0.13359137839416452
  (0, 24957)	0.09146351035625488
  (0, 43131)	0.15491381382108724
  (0, 2051)	0.4159373110833513
  (0, 46617)	0.46474144146326174
  (0, 24361)	0.10943293956646011
  :	:
  (19997, 13221)	0.17897321248705433
  (19997, 37592)	0.15166031585027698
  (19997, 45919)	0.1821847673691749
  (19997, 46772)	0.14244710628029003
  

In [119]:
cosine_sim=cosine_similarity(tfidf_matrix, tfidf_matrix) #같은 tfidf_matrix끼리의 유도이므로 전치행렬이 나옴
print('코사인 유사도 연산 결과 크기: ', cosine_sim.shape)
print(cosine_sim)

코사인 유사도 연산 결과 크기:  (20000, 20000)
[[1.         0.01575748 0.         ... 0.         0.         0.        ]
 [0.01575748 1.         0.04907345 ... 0.         0.         0.        ]
 [0.         0.04907345 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.08375766]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.08375766 0.         1.        ]]


In [120]:
#기존 데이터프레임으로부터 영화의 제목을 key, 영화의 인덱스를 value로 하는 딕셔너리를 만든다.
title_to_index=dict(zip(data['title'], data.index))

In [121]:
#가장 코사인 유사도가 높은 영화 10개를 찾아내는 함수
def get_recommendations(title, cosine_sim=cosine_sim): #코사인 유사도는 이미 정해져 있으므로 title만 불러온다.
  #해당 영화의 인덱스를 알아낸다.
  idx=title_to_index[title]
  #해당 영화와 모든 영화와의 유사도를 가져온다.(리스트는 (index, cosine유사도))
  sim_scores=list(enumerate(cosine_sim[idx])) #해당 영화의 인덱스를 넣으면 해당 영화와 다른 모든 영화들 간의 유사도가 나온다.
  #유사도에 따라 영화들을 정렬한다.
  sim_scores=sorted(sim_scores, key=lambda x:x[1], reverse=True) #sim_socres에는 첫 인덱스에 인덱스, 두번째 인덱스에 유사도가 들어가있다.
  #가장 유사한 10개의 영화를 받아온다.
  sim_scores=sim_scores[1:11] #정렬을 이미 했으므로 맨 앞부터 10개만 가져온다.
  #가장 유사한 10개 영화의 인덱스를 얻는다.
  movie_indexes=[idx[0] for idx in sim_scores] #sim_scores의 첫 인덱스에 해당 인덱스가 적혀있다.
  #가장 유사한 10개 영화의 제목을 리턴한다.
  return data['title'].iloc[movie_indexes] #iloc은 iloc[row, col]으로 해당 좌표를 추출한다. 슬라이싱으로 여러 범위도 가능. 여기서는 sim_scores에 기록된 영화들만 추출하는 기능


In [122]:
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object



*   유클리드 거리는 두 점 사이의 직선 거리를 구한다.(=피타고라스 정리)



In [127]:
docs=['바나나 바나나 사과 사과 사과 좋아요',
      '저는 바나나 저는 사과 저는 사과 좋아요',
      '저는 바나나 바나나 저는 사과 좋아요',
      '바나나 사과 좋아요']

vocab=list(set(w for doc in docs for w in doc.split())) #set은 겹치지 않은 것만 추가

def tf(t, d):
  return d.count(t)

result=[]

for doc in docs:
  result.append([])
  d=doc
  for word in vocab:
    t=word
    result[-1].append(tf(t, d))
tf_=pd.DataFrame(result, columns=vocab)
tf_
tf=np.array(tf_)

In [128]:
import numpy as np

def dist(x,y):
  return np.sqrt(np.sum((x-y)**2)) # x와y의 차를 제곱하여 모두 더한 값에 제곱근

print('문서 1과 문서의 거리 : ', dist(tf[0],tf[3]))
print('문서 2과 문서의 거리 : ', dist(tf[1],tf[3]))
print('문서 3과 문서의 거리 : ', dist(tf[2],tf[3]))

문서 1과 문서의 거리 :  2.23606797749979
문서 2과 문서의 거리 :  3.1622776601683795
문서 3과 문서의 거리 :  2.23606797749979


자카드 유사도 : A와 B 두개의 집합의 합집합에서 교집합의 비율 --> 합집합에 교집합이 포함되어 있으므로 단순하게 (합집합의 길이 / 교집합의 길이)를 구하면 된다.

In [129]:
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"

#토큰화
tokenized_doc1=doc1.split()
tokenized_doc2=doc2.split()

print('문서 1: ', tokenized_doc1)
print('문서 2: ', tokenized_doc2)

문서 1:  ['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
문서 2:  ['apple', 'banana', 'coupon', 'passport', 'love', 'you']


In [130]:
#문서 1과 문서 2의 합집합 구하기
doc_union=set(tokenized_doc1).union(set(tokenized_doc2)) #doc1의 토큰들을 중복x인 경우와 doc2의 토큰들을 중복x로 한 경우의 합집합
doc_intersection=set(tokenized_doc1).intersection(set(tokenized_doc2)) #마찬가지로 교집합
print('합집합 : ', doc_union)
print('교집합 : ', doc_intersection)

합집합 :  {'you', 'apple', 'banana', 'everyone', 'watch', 'card', 'likey', 'like', 'coupon', 'holder', 'passport', 'love'}
교집합 :  {'banana', 'apple'}


In [132]:
print('자카드 유사도 : ', len(doc_intersection)/len(doc_union))

자카드 유사도 :  0.16666666666666666
